# Momentum ETF Performance

In [2]:
import pandas as pd
import numpy as np
from pandas_datareader import DataReader as pdr
import plotly.graph_objects as go
import statsmodels.api as sm

In [19]:
# Pull data
ticker_list = ['MTUM','PDP','XMMO','ONEO','VFMO','JMOM','SPMO','QMOM','FDMO','MMTM']

# Pull the data from Yahoo
df = pdr(ticker_list, "yahoo", start=2000)
df = df.loc[:,('Adj Close',slice(None))].droplevel('Attributes', axis=1)
df = df.resample('M').last()
df = df.pct_change()
df.index = df.index.to_period("M")
df

Symbols,MTUM,PDP,XMMO,ONEO,VFMO,JMOM,SPMO,QMOM,FDMO,MMTM
Date,,,,,,,,,,
2005-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-04,NaN,NaN,-0.036806,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-05,NaN,NaN,0.067772,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-06,NaN,NaN,0.043214,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-07,NaN,NaN,0.034304,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-06,-0.063900,-0.111778,-0.119705,-0.092563,-0.095816,-0.083604,-0.081897,-0.119669,-0.078737,-0.081897
2022-07,0.051698,0.109692,0.125668,0.086619,0.083573,0.101321,0.079079,0.095048,0.097747,0.085748
2022-08,-0.019942,-0.020114,-0.031968,-0.024062,-0.003931,-0.033766,-0.029546,0.029649,-0.026617,-0.030198


In [20]:
# Add factors
ff3 = pdr('F-F_Research_Data_Factors','famafrench', start=1900)[0]/100
Mom = pdr('F-F_Momentum_Factor','famafrench', start=1900)[0]/100
Mom.columns = ['Mom']
ff = ff3.join(Mom)
df = df.join(ff)

In [27]:
stats = pd.DataFrame(dtype=float, columns=pd.MultiIndex.from_product([['CAPM','FF4'],['alpha','beta','t_alpha','t_beta','nobs']]), index=ticker_list)
stats

CAPM                            FF4                         
     alpha beta t_alpha t_beta nobs alpha beta t_alpha t_beta nobs
MTUM   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
PDP    NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
XMMO   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
ONEO   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
VFMO   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
JMOM   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
SPMO   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
QMOM   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
FDMO   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
MMTM   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN

In [28]:
# Run the performance evaluations
for tick in ticker_list:
    # Market-model
    mm = sm.OLS(df[tick]-df['RF'], sm.add_constant(df['Mkt-RF']),missing='drop').fit()
    stats.loc[tick, ('CAPM','alpha')] = mm.params[0]
    stats.loc[tick, ('CAPM','beta')]  = mm.params[1]
    stats.loc[tick, ('CAPM','t_alpha')] = mm.tvalues[0]
    stats.loc[tick, ('CAPM','t_beta')]  = mm.tvalues[1]
    stats.loc[tick, ('CAPM','nobs')]  = mm.nobs

    # Fama-French 4-factor model
    ff4 = sm.OLS(df[tick]-df['RF'], sm.add_constant(df[['Mkt-RF','SMB','HML','Mom']]),missing='drop').fit()
    stats.loc[tick, ('FF4','alpha')] = ff4.params[0]
    stats.loc[tick, ('FF4','beta')]  = ff4.params[1]
    stats.loc[tick, ('FF4','t_alpha')] = ff4.tvalues[0]
    stats.loc[tick, ('FF4','t_beta')]  = ff4.tvalues[1]  
    stats.loc[tick, ('FF4','nobs')]    = ff4.nobs 


In [29]:
stats

CAPM                                             FF4            \
         alpha      beta   t_alpha     t_beta   nobs     alpha      beta   
MTUM  0.001594  0.892024  0.909469  22.900051  113.0 -0.000345  1.000916   
PDP  -0.000862  1.013292 -0.575796  32.527490  186.0 -0.001964  1.085595   
XMMO  0.000969  1.035783  0.620356  30.364277  210.0  0.000314  1.052896   
ONEO -0.002518  1.024140 -1.656175  32.972446   81.0 -0.002255  1.004034   
VFMO  0.000768  1.000465  0.256869  18.694444   55.0 -0.000032  1.019577   
JMOM  0.000475  0.979312  0.264040  30.028904   58.0 -0.000745  1.042721   
SPMO  0.001943  0.855522  0.973252  20.741063   83.0  0.000568  0.980959   
QMOM -0.000448  1.092740 -0.102014  12.173835   81.0 -0.001600  1.185257   
FDMO -0.000294  0.936847 -0.185096  29.835235   72.0 -0.001686  1.021410   
MMTM  0.000575  0.920682  0.547518  38.897387  119.0 -0.000269  0.980108   

                                  
       t_alpha     t_beta   nobs  
MTUM -0.287829  33.551646  113.0  
PDP  -1.591041  37.435900  186.0  
XMMO  0.233389  31.637707  210.0  
ONEO -1.840685  34.417390   81.0  
VFMO -0.017472  26.019103   55.0  
JMOM -0.600847  39.817334   58.0  
SPMO  0.387942  27.816273   83.0  
QMOM -0.461618  14.357937   81.0  
FDMO -1.515210  40.191980   72.0  
MMTM -0.287152  41.458763  119.0

In [32]:
# Sort to find the highest historical alpha
stats.sort_values(by=[('CAPM','t_alpha')], ascending=False)

CAPM                                             FF4            \
         alpha      beta   t_alpha     t_beta   nobs     alpha      beta   
SPMO  0.001943  0.855522  0.973252  20.741063   83.0  0.000568  0.980959   
MTUM  0.001594  0.892024  0.909469  22.900051  113.0 -0.000345  1.000916   
XMMO  0.000969  1.035783  0.620356  30.364277  210.0  0.000314  1.052896   
MMTM  0.000575  0.920682  0.547518  38.897387  119.0 -0.000269  0.980108   
JMOM  0.000475  0.979312  0.264040  30.028904   58.0 -0.000745  1.042721   
VFMO  0.000768  1.000465  0.256869  18.694444   55.0 -0.000032  1.019577   
QMOM -0.000448  1.092740 -0.102014  12.173835   81.0 -0.001600  1.185257   
FDMO -0.000294  0.936847 -0.185096  29.835235   72.0 -0.001686  1.021410   
PDP  -0.000862  1.013292 -0.575796  32.527490  186.0 -0.001964  1.085595   
ONEO -0.002518  1.024140 -1.656175  32.972446   81.0 -0.002255  1.004034   

                                  
       t_alpha     t_beta   nobs  
SPMO  0.387942  27.816273   83.0  
MTUM -0.287829  33.551646  113.0  
XMMO  0.233389  31.637707  210.0  
MMTM -0.287152  41.458763  119.0  
JMOM -0.600847  39.817334   58.0  
VFMO -0.017472  26.019103   55.0  
QMOM -0.461618  14.357937   81.0  
FDMO -1.515210  40.191980   72.0  
PDP  -1.591041  37.435900  186.0  
ONEO -1.840685  34.417390   81.0

In [36]:
alpha = stats.loc['SPMO',('CAPM','alpha')]*12
print(f'{alpha: .2%}')

 2.33%


In [37]:
ff4 = sm.OLS(df['SPMO']-df['RF'], sm.add_constant(df[['Mkt-RF','SMB','HML','Mom']]),missing='drop').fit()
print(ff4.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.919
Model:                            OLS   Adj. R-squared:                  0.915
Method:                 Least Squares   F-statistic:                     222.0
Date:                Thu, 27 Oct 2022   Prob (F-statistic):           8.82e-42
Time:                        16:16:47   Log-Likelihood:                 245.06
No. Observations:                  83   AIC:                            -480.1
Df Residuals:                      78   BIC:                            -468.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0006      0.001      0.388      0.6